# Agents

- People sometimes have unrealistic expectations for what a LLM can do.
  - Complex arithmetic operations with numbers with many digits
  - Solving more complex equations, integrals, differential equations
  - Retrieving very specific information e.g. what is the 7th country in Africa by GDP?
- But a more useful way to think of LLMs is to think of them as **reasoning engines**.
- So they might use some background knowledge to which they were exposed at training time, but we shouldn't rely 100% on them.
- On the other hand, for a variety of tasks there are great specific tools that we can use!
- **Agents** combine LLMs with different tools that can be used to solve your tasks: internet browsers, symbolic engines, knowledge bases, etc.
- LangChain comes with a huge list of Tools you can use to build your own Agents! Some examples:
  - **Shell (bash)** — give access to a shell in a safe environment
  - **Bing Search / DuckDuckGo Search / Google Search /** ... — Search the web using different search engines
  - **ChatGPT Plugins** — use ChatGPT plugins to interact with the world
  - **Dall-E Image Generator** — generate images from a prompt synthesized using an OpenAI LLM
  - **PubMed** — Search the PubMed database for biomedical literature
  - **Wikipedia** — Search Wikipedia for articles, media, and other contents
  - **Wolfram Alpha** — Use Wolfram Alpha to solve equations, ... or access knowledge graph information
  - **Yahoo Finance** — Get stock information from Yahoo Finance

In [1]:
import os
from pprint import pprint
from dotenv import load_dotenv

In [2]:
# 1. Copy .env.example file as .env: `cp .env.example .env`
# 2. Open .env file and set all the env variables
load_dotenv(".env")
OPENAI_KEY = os.getenv("OPENAI_KEY")
assert OPENAI_KEY, "Please set your OPENAI_KEY environment variable."

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool


In [4]:
# The Chat Model
chat_model = ChatOpenAI(
    openai_api_key=OPENAI_KEY,
    model_name="gpt-4",
    temperature=0,
    model_kwargs={"top_p":1},
)

In [5]:
tools = load_tools(["llm-math", "wikipedia"], llm=chat_model) + [YahooFinanceNewsTool()]

In [6]:
agent= initialize_agent(
    tools, 
    chat_model, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [7]:
question = """
How much is (3 * 334 / 17 + 9) ** 2?
"""
agent.run(question)



> Entering new AgentExecutor chain...
Thought: This is a mathematical calculation. I can use the Calculator tool to solve it.
Action:
```
{
  "action": "Calculator",
  "action_input": "(3 * 334 / 17 + 9) ** 2"
}
```
Observation: Answer: 4616.003460207612
Thought:The calculation has been completed and the result is approximately 4616.003460207612. This is the final answer. 
Final Answer: 4616.003460207612

> Finished chain.


'4616.003460207612'

In [8]:
question = """
The 2023 Spanish general election was held on Sunday, 23 July 2023.
Which party won the most seats?
"""
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I need to find information about the 2023 Spanish general election. The best tool for this is Wikipedia.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "2023 Spanish general election"
}
```
Observation: Page: 2023 Spanish general election
Summary: The 2023 Spanish general election was held on Sunday, 23 July 2023, to elect the 15th Cortes Generales of the Kingdom of Spain. All 350 seats in the Congress of Deputies were up for election, as well as 208 of 265 seats in the Senate.
The second government of Pedro Sánchez formed after the November 2019 Spanish general election consisted of a left-wing coalition between the Spanish Socialist Workers' Party (PSOE) and Unidas Podemos, the country's first such nationwide government since the times of the Second Spanish Republic. The government's tenure was quickly overshadowed by the outbreak of the COVID-19 pandemic in March 2020, along with its political and economic consequences. Thes

In [9]:
question = """
What happens today with Microsoft stocks?
"""

agent.run(question)



> Entering new AgentExecutor chain...
Thought: I need to find the latest financial news about Microsoft. I can use the yahoo_finance_news tool for this.
Action:
```
{
  "action": "yahoo_finance_news",
  "action_input": "MSFT"
}
```
Observation: Why Amazon Stock Dropped on Tuesday
Shares of Amazon.com (NASDAQ: AMZN) slid 3% through 12:30 p.m. ET on Tuesday after Reuters -- in an exclusive report -- warned that British regulators may launch an antimonopoly investigation into the company's dominance of the cloud computing marketplacej.  Microsoft (NASDAQ: MSFT) -- also reported to be part of the probe -- saw its shares slide a similar 2.5%.  As Reuters reports, the British media regulator "Ofcom" (short for the Office of Communications) has actually been looking into Amazon's and Microsoft's dominance in cloud computing since April -- but is now preparing to issue an official report on Thursday.
Thought:The latest news indicates that Microsoft's stocks have slid by 2.5% due to a potenti

"Microsoft's stocks have dropped by 2.5% today due to a potential antimonopoly investigation by British regulators into the company's dominance in the cloud computing market."